# Lab 3: Exploring Hugging Face Transformers

## Objectives
- Learn to use the Hugging Face `transformers` library
- Explore different pre-trained models
- Understand model loading and inference
- Perform basic NLP tasks using transformers

## Prerequisites
- Completed Labs 1 and 2
- Basic understanding of transformers architecture

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    pipeline
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Text Classification with Transformers

In [ ]:
def analyze_sentiment(text):
    classifier = pipeline("sentiment-analysis")
    result = classifier(text)
    return result[0]

# Test with different texts
texts = [
    "I love working with transformers, they're amazing!",
    "This code is quite confusing and difficult to understand.",
    "The weather is nice today."
]

for text in texts:
    sentiment = analyze_sentiment(text)
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment['label']}, Score: {sentiment['score']:.3f}\n")

## 2. Text Generation with GPT-2

In [ ]:
def generate_text(prompt, max_length=100):
    generator = pipeline('text-generation', model='gpt2')
    result = generator(prompt, max_length=max_length, num_return_sequences=1)
    return result[0]['generated_text']

# Test text generation
prompt = "In the future, artificial intelligence will"
generated_text = generate_text(prompt)
print(f"Generated text:\n{generated_text}")

## 3. Named Entity Recognition (NER)

In [ ]:
def extract_entities(text):
    ner = pipeline("ner")
    entities = ner(text)
    
    # Group overlapping entities
    merged_entities = []
    current_entity = None
    
    for entity in entities:
        if current_entity is None:
            current_entity = entity
        elif entity['start'] <= current_entity['end']:
            if entity['score'] > current_entity['score']:
                current_entity = entity
        else:
            merged_entities.append(current_entity)
            current_entity = entity
    
    if current_entity is not None:
        merged_entities.append(current_entity)
    
    return merged_entities

# Test NER
text = "Apple CEO Tim Cook announced new AI features at their headquarters in Cupertino, California."
entities = extract_entities(text)

print("Found entities:")
for entity in entities:
    print(f"- {entity['word']}: {entity['entity']} (confidence: {entity['score']:.3f})")

## 4. Zero-shot Classification

In [ ]:
def classify_text(text, labels):
    classifier = pipeline("zero-shot-classification")
    result = classifier(text, labels)
    return result

# Test zero-shot classification
text = "This new smartphone has an amazing camera and great battery life."
labels = ["technology", "photography", "battery", "design"]

result = classify_text(text, labels)
print(f"Text: {text}\n")
print("Classifications:")
for label, score in zip(result['labels'], result['scores']):
    print(f"- {label}: {score:.3f}")

## 5. Custom Model Loading and Inference

In [ ]:
def load_custom_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

def predict_with_custom_model(text, model_name="distilbert-base-uncased-finetuned-sst-2-english"):
    tokenizer, model = load_custom_model(model_name)
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    return predictions.detach().numpy()

# Test custom model
text = "This is one of the best courses I've ever taken!"
predictions = predict_with_custom_model(text)
print(f"Prediction scores: {predictions[0]}")

## Exercises

1. Implement a multi-lingual sentiment analysis system using Hugging Face models
2. Create a text summarization pipeline using a pre-trained model
3. Build a question-answering system using the transformers pipeline
4. Experiment with different model architectures and compare their performance

## Next Steps
- Explore model fine-tuning
- Learn about model quantization and optimization
- Try more advanced NLP tasks
- Experiment with different model architectures